In [1]:
import pandas as pd

import webbrowser  


In [2]:
for_mapbox = pd.DataFrame(columns=['name', 'latitude', 'longitude', 'phone_number', 'address', 'additional_info', 'type'])

In [3]:
se = pd.read_csv('Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv')
se = se.sort_values(by='COMMUNITY AREA NAME')


import geopandas as gpd
df = gpd.read_file('Boundaries - Neighborhoods.geojson')

df['center'] = df['geometry'].apply(lambda x: str((x.centroid)).strip('0 POINT '))

df['longitude'] = df.center.apply(lambda x: float(x.split(" ")[0].strip("(")))
df['latitude'] = df.center.apply(lambda x: float(x.split(" ")[1].strip(")")))

df = df[['pri_neigh', 'sec_neigh', 'longitude', 'latitude']]
df = df.sort_values(by='pri_neigh').head(80)

se = se.merge(df, left_on='COMMUNITY AREA NAME', right_on='pri_neigh')

In [4]:
se.to_csv('socioeconomic.csv')

In [5]:
housing = pd.read_csv('Affordable_Rental_Housing_Developments.csv')
housing = housing[['Property Name', 'Latitude', 'Longitude', 'Phone Number', 'Address', 'Property Type']]
housing.columns=['name', 'latitude', 'longitude', 'phone_number','address', 'additional_info']
housing['type'] = "Affordable Housing"
housing.additional_info = "Address: " + housing.additional_info

housing.head()

,name,latitude,longitude,phone_number,address,additional_info,type
0,Winthrop Apts.,41.995015,-87.658516,773-477-7070,6214 N. Winthrop Ave.,Address: Multifamily,Affordable Housing
1,Resurrection Homes,41.855133,-87.703067,312-248-8355,1910 S. Albany Ave.,Address: Multifamily,Affordable Housing
2,Renaissance St. Luke Greenview Place,41.940531,-87.666383,773-525-1501,1501 W. Melrose St.,Address: Senior,Affordable Housing
3,All Saints Senior Residence,41.681590,-87.622628,312-655-7440,11701 S. State St.,Address: Senior,Affordable Housing
4,Victory Center of Roseland,41.703891,-87.620771,773-468-6400,10450 S. Michigan Ave.,Address: Senior,Affordable Housing


In [6]:
grocery = pd.read_csv('IL.csv')
grocery = grocery[grocery.CITY.isin(['Chicago', 'Chiacgo', 'CHICAGO'])]
grocery['additional_info'] = "Accepts SNAP/EBT"
grocery['phone_number'] = ""
grocery = grocery[['STORE_NAME', 'latitude', 'longitude', 'phone_number', 'ADDRESS', 'additional_info']]
grocery.columns=['name', 'latitude', 'longitude', 'phone_number','address', 'additional_info']
grocery.head()

,name,latitude,longitude,phone_number,address,additional_info
6,SOUTH SIDE MARATHON INC,41.779179,-87.693428,,2800 W 63rd St,Accepts SNAP/EBT
8,MORSE FOOD AND MEAT MARKET,42.007919,-87.666962,,1430 W Morse Ave,Accepts SNAP/EBT
9,81ST AND RACINE CITGO INC,41.746632,-87.653679,,8106 S Racine Ave,Accepts SNAP/EBT
12,Pershing Center,41.823582,-87.632385,,243 W Pershing Rd,Accepts SNAP/EBT
21,El Barakah Supermarket,41.763927,-87.560486,,7300 S South Shore Dr,Accepts SNAP/EBT


In [7]:
libraries = pd.read_csv('Libraries_-_Locations__Hours_and_Contact_Information.csv')
libraries['latitude'] = libraries.LOCATION.apply(lambda x: float(x.strip("()").split(",")[0]))
libraries['longitude'] = libraries.LOCATION.apply(lambda x: float(x.strip("()").split(",")[1]))
libraries = libraries[['NAME ', 'latitude', 'longitude', 'PHONE', 'ADDRESS', 'HOURS OF OPERATION']]
libraries.columns=['name', 'latitude', 'longitude', 'phone_number', 'address', 'additional_info']
libraries.additional_info = "Hours: " + libraries.additional_info
libraries['type'] = "Library"

libraries.head()

,name,latitude,longitude,phone_number,address,additional_info,type
0,South Shore,41.761555,-87.562084,Closed for Construction,2505 E. 73rd St.,NaN,Library
1,Albany Park,41.975456,-87.714090,(773) 539-5450,3401 W. Foster Ave.,"Hours: Sun., Closed; Mon. & Wed., 10-6; Tue. &...",Library
2,Altgeld,41.654730,-87.602230,(312) 747-3270,13281 S. Corliss Ave.,"Hours: Sun., Closed; Mon. & Wed., Noon-8; Tue....",Library
3,Archer Heights,41.801214,-87.726491,(312) 747-9241,5055 S. Archer Ave.,"Hours: Sun., Closed; Mon. & Wed., Noon-8; Tue....",Library
4,Austin,41.889272,-87.765712,(312) 746-5038,5615 W. Race Ave.,"Hours: Sun., Closed; Mon. & Wed., Noon-8; Tue....",Library


In [8]:
spqr = pd.ExcelFile('SQRP_18_19.xls')
elem = pd.read_excel(spqr, "elem", header=0, skiprows=1)
high = pd.read_excel(spqr, "high")
combo = pd.read_excel(spqr, "combo")
option = pd.read_excel(spqr, "option")

all_schools = elem
for s in ['high', 'combo', 'option']:
    all_schools = all_schools.append(pd.read_excel(spqr, s, header=0))
    print(all_schools.shape)

(592, 104)
(621, 179)
(663, 182)


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [9]:
all_schools2 = all_schools[['School ID', 'SY 2018-2019 SQRP Rating']]
all_schools2.columns = ['School_ID', 'additional_info']
all_schools2 = all_schools2.dropna(subset=['School_ID'])
all_schools2.School_ID = all_schools2.School_ID.apply(lambda x: int(x))

In [14]:
s = pd.read_csv('Chicago_Public_Schools_-_School_Profile_Information_SY1617 (1).csv')
schools = s.merge(all_schools2, on='School_ID', how='left')
schools = schools[['Long_Name', 'School_Latitude', 'School_Longitude', 'Phone', 'Address', 'additional_info']]
schools.columns=['name', 'latitude', 'longitude', 'phone_number', 'address', 'additional_info']
schools['type'] = "School"
schools['additional_info'] = "SQRP Rating: " + schools.additional_info

schools.head()

,name,latitude,longitude,phone_number,address,additional_info,type
0,Harriet E Sayre Elementary Language Academy,41.914151,-87.798721,7735343351,1850 N NEWLAND AVE,SQRP Rating: Level 2+,School
1,Ronald E McNair Elementary School,41.897819,-87.746731,7735348980,4820 W WALTON ST,SQRP Rating: Level 1+,School
2,Charles N Holden Elementary School,41.838027,-87.653792,7735357200,1104 W 31ST ST,SQRP Rating: Level 1+,School
3,Acero Charter Schools - SPC Daniel Zizumbo,41.810135,-87.730495,7735793470,4248 W 47TH ST,SQRP Rating: Level 1,School
4,John B Murphy Elementary School,41.950078,-87.716832,7735345223,3539 W GRACE ST,SQRP Rating: Level 2+,School


In [22]:
for_mapbox = for_mapbox.append(housing)
for_mapbox = for_mapbox.append(libraries)
for_mapbox = for_mapbox.append(schools)
for_mapbox = for_mapbox.append(grocery)

for_mapbox = for_mapbox.fillna('')
for_mapbox.columns = ['name', 'latitude', 'longitude', 'phone_number','address', 'additional_info', 'type']

In [23]:
for_mapbox.head()

,name,latitude,longitude,phone_number,address,additional_info,type
0,Address: Multifamily,6214 N. Winthrop Ave.,41.995015,-87.658516,Winthrop Apts.,773-477-7070,Affordable Housing
1,Address: Multifamily,1910 S. Albany Ave.,41.855133,-87.703067,Resurrection Homes,312-248-8355,Affordable Housing
2,Address: Senior,1501 W. Melrose St.,41.940531,-87.666383,Renaissance St. Luke Greenview Place,773-525-1501,Affordable Housing
3,Address: Senior,11701 S. State St.,41.681590,-87.622628,All Saints Senior Residence,312-655-7440,Affordable Housing
4,Address: Senior,10450 S. Michigan Ave.,41.703891,-87.620771,Victory Center of Roseland,773-468-6400,Affordable Housing


In [18]:
for_mapbox.to_csv('for_mapbox.csv')